In [ ]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import time
import cv2
from PIL import Image

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
import save

# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [ ]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 81

config = InferenceConfig()
# config.display()

In [ ]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

In [ ]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']
print(class_names.index('bottle'))

In [ ]:
img_dir = '/srv/glusterfs/rheiki/data/videos/interview/frames'
img_ext = '.png'
file_names = []
for f in os.listdir(img_dir):
    if f.endswith(img_ext):
        file_names.append(f)
folder_path = '/srv/glusterfs/rheiki/data/videos/interview/crops'
padding_size = 10
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

In [ ]:
print("Total number of images: {}".format(len(file_names)))
for index, file_name in enumerate(sorted(file_names)):
    img_path = os.path.join(folder_path, file_name)
    if os.path.isfile(img_path):
        print("Skipping image {}. Image already exists.".format(file_name))
        continue        
    image = skimage.io.imread(os.path.join(img_dir, file_name))
    # Run detection
    result = model.detect([image], verbose=0)
    r = result[0]
    if not len(np.where(r['class_ids'] == 40)[0]):
        print("Skipping image {}. Bottle not found.".format(file_name))
        continue
    bottle_index = np.where(r['class_ids'] == 40)[0][0]
    r['masks'] = r['masks'][:, :, bottle_index:bottle_index + 1]
    r['rois'] = r['rois'][bottle_index:bottle_index + 1]
    r['class_ids'] = r['class_ids'][bottle_index:bottle_index + 1]
    left, top, right, bottom = r['rois'][0]
    img = Image.fromarray(image, 'RGB')
    cropped = img.crop((top - padding_size, left - padding_size, bottom + padding_size, right + padding_size))
    cropped.save(img_path, "PNG")
    print("Saved image {img_name}. {no_left} images remaining.".format(img_name=new_name, no_left=len(file_names) - index - 1))

## Next cells are used only for debugging.

In [ ]:
# Visualize results
image = skimage.io.imread(os.path.join(img_dir, sorted(file_names)[456]))
# Run detection
result = model.detect([image], verbose=0)
r = result[0]
if not len(np.where(r['class_ids'] == 40)[0]):
    print("Skip")
bottle_index = np.where(r['class_ids'] == 40)[0][0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])

In [ ]:
r['masks'] = r['masks'][:, :, bottle_index:bottle_index + 1]
r['rois'] = r['rois'][bottle_index:bottle_index + 1]
r['class_ids'] = r['class_ids'][bottle_index:bottle_index + 1]

In [ ]:
left, top, right, bottom = r['rois'][0]
img = Image.fromarray(image, 'RGB')
w, h = img.size
img.show()
cropped = img.crop((top - 10, left - 10, bottom + 10, right + 10))
file_name = file_names[0]
img_path = os.path.join(folder_path, file_name)
cropped.save(img_path)
#save.save_instance(image=image, boxes=r['rois'], masks=r['masks'], class_ids=r['class_ids'], class_names=class_names, scores=r['scores'], show_bbox=True, show_mask=False, save_path=img_path)